In [1]:
# !pip install tqdm
# !pip install torch torchvision torchaudio
# !pip install fastai
# !pip install tensorboardX
# !pip install scikit-learn
# !pip install "git+https://github.com/ncullen93/torchsample.git#egg=torchsample"
# !pip install nibabel
# !pip install nbdev

In [2]:
import shutil
import os
import time
from datetime import datetime
import argparse
import numpy as np
from tqdm import tqdm
from nbdev import *

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchsample.transforms import RandomRotate, RandomTranslate, RandomFlip, ToTensor, Compose, RandomAffine
from torchvision import transforms
import torch.nn.functional as F
from tensorboardX import SummaryWriter

from dataloader import MRDataset
import model

from sklearn import metrics
from fastai.vision.all import *

In [3]:
# parser.add_argument('-t', '--task', type=str, required=True,
#                     choices=['abnormal', 'acl', 'meniscus'])
# parser.add_argument('-p', '--plane', type=str, required=True,
#                     choices=['sagittal', 'coronal', 'axial'])
# parser.add_argument('--prefix_name', type=str, required=True)
# parser.add_argument('--augment', type=int, choices=[0, 1], default=1)
# parser.add_argument('--lr_scheduler', type=str,
#                     default='plateau', choices=['plateau', 'step'])
# parser.add_argument('--gamma', type=float, default=0.5)
# parser.add_argument('--epochs', type=int, default=50)
# parser.add_argument('--lr', type=float, default=1e-5)
# parser.add_argument('--flush_history', type=int, choices=[0, 1], default=0)
# parser.add_argument('--save_model', type=int, choices=[0, 1], default=1)
# parser.add_argument('--patience', type=int, default=5)
# parser.add_argument('--log_every', type=int, default=100)
# args = parser.parse_args()

class Args:
    def __init__(self):
        self.task = "abnormal"
        self.plane = "sagittal"
        self.prefix_name = "Test"
        self.augment = 1
        self.lr_scheduler = "plateau"
        self.gamma = 0.5
        self.epochs = 50
        self.lr = 1e-5
        self.flush_history = 0
        self.save_model =1
        self.patience = 5
        self.log_every = 100
        
args = Args()

In [4]:
args.task

'abnormal'

In [5]:
log_root_folder = "./logs/{0}/{1}/".format(args.task, args.plane)
if args.flush_history == 1:
    objects = os.listdir(log_root_folder)
    for f in objects:
        if os.path.isdir(log_root_folder + f):
            shutil.rmtree(log_root_folder + f)

now = datetime.now()
logdir = log_root_folder + now.strftime("%Y%m%d-%H%M%S") + "/"
os.makedirs(logdir)

writer = SummaryWriter(logdir)

augmentor = Compose([
    transforms.Lambda(lambda x: torch.Tensor(x)),
    RandomRotate(25),
    RandomTranslate([0.11, 0.11]),
    RandomFlip(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1, 1).permute(1, 0, 2, 3)),
])

train_dataset = MRDataset('./data/', args.task,
                          args.plane, transform=augmentor, train=True)
# train_loader = torch.utils.data.DataLoader(
#     train_dataset, batch_size=1, shuffle=True, num_workers=11, drop_last=False)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=7, drop_last=False)

validation_dataset = MRDataset(
    './data/', args.task, args.plane, train=False)
# validation_loader = torch.utils.data.DataLoader(
#     validation_dataset, batch_size=1, shuffle=-True, num_workers=11, drop_last=False)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False, num_workers=3, drop_last=False, device="cuda:0")
validation_loader = DataLoader(validation_dataset, batch_size=1, shuffle=False, num_workers=3, drop_last=False, device="cuda:0")



mrnet = model.MRNet()
bceloss = nn.BCEWithLogitsLoss()

./data/train-abnormal.csv


In [6]:
dls = DataLoaders(train_loader, validation_loader)

In [7]:
learn = Learner(dls, mrnet, loss_func=bceloss)

In [8]:
DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=7, drop_last=False)

In [9]:
#learn.fine_tune(1)